# Migros API to extract Food Data

This is for extracting the data from the Migros API to get food/logistic data. There will be several categories of this speicified later on.

In [ ]:
hackzurich2020
uhSyJ08KexKn4ZFS

In [7]:
import requests
import json

In [ ]:
#Products API, with verbosity = id and limit=2000, then go through the page ids of all products

offset = 0
url = "https://hackzurich-api.migros.ch/hack/recipe/recipes_de/_search"
response = requests.get(url, auth=('hackzurich2020', 'uhSyJ08KexKn4ZFS'))
response_json = response.json()
with open('migros_recipes.json', 'w') as fp:
    json.dump(response_json, fp)
